In [1]:
import pandas as pd, numpy as np

In [2]:
ro=['5765358043206','9812808043220','9576658043223','1699958043227','7225068043228','265208043229']
hu=['8073718043234','2087988043232','6247548043235']

In [3]:
stations=[]
for i in ro+hu:
    stations.append(pd.read_csv(i+'stn+.txt',delimiter= '+',skiprows=2,header=None))
    print(i)

5765358043206
9812808043220
9576658043223
1699958043227
7225068043228
265208043229
8073718043234
2087988043232
6247548043235


In [4]:
station=pd.concat(stations)

In [5]:
station=station.drop_duplicates()

In [53]:
station[2]=station[2].str.strip()
station[3]=station[3].str.strip()

In [195]:
dfs=[]
for i in ro+hu:
    df=pd.read_csv(i+'dat.txt',delimiter= '\s+')
    df=df.replace({'******':np.nan,'*****':np.nan,'****':np.nan,'***':np.nan,'**':np.nan,'*':np.nan})
    df['time']=pd.to_datetime(df['YR--MODAHRMN'],format='%Y%m%d%H%M')
    df=df[['time','USAF','SPD','TEMP','PCP06','SD','MW','AW','W']]
    df['TEMP']=(pd.to_numeric(df['TEMP'], errors='coerce')-32)*5/9
    df['SPD']=pd.to_numeric(df['SPD'], errors='coerce')*1.61
    df['PCP06']=pd.to_numeric(df['PCP06'], errors='coerce')*25.4
    df['SD']=pd.to_numeric(df['SD'], errors='coerce')*25.4
    dfs.append(df)
    print(i)

5765358043206
9812808043220
9576658043223
1699958043227
7225068043228
265208043229


C:\ProgramData\Anaconda3\envs\plotly\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning:

Columns (4,11,12,21,22,23,25) have mixed types. Specify dtype option on import or set low_memory=False.



8073718043234
2087988043232


C:\ProgramData\Anaconda3\envs\plotly\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning:

Columns (4,11,12,20,21,22,23,24,25) have mixed types. Specify dtype option on import or set low_memory=False.



6247548043235


In [200]:
dfs=pd.concat(dfs)

In [201]:
dfs=dfs.drop_duplicates()

In [343]:
dfs[dfs['USAF']==154550].year.unique()

array([1974, 1978, 1979, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987,
       1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1997, 1998, 1999,
       2000, 2001, 2002, 2003, 2004, 2009, 2012, 2014, 2015, 2016, 2017,
       2018, 2019], dtype=int64)

In [202]:
dfs.to_csv('data/all.csv')

In [301]:
dfs=pd.read_csv('data/all.csv')

In [302]:
dfs['time']=pd.to_datetime(dfs['time'])
dfs['year']=dfs['time'].dt.year
dfs['month']=dfs['time'].dt.month
dfs['day']=dfs['time'].dt.day
dfs['hour']=dfs['time'].dt.hour

In [303]:
df=dfs.groupby(['USAF','year','month','day','hour'])[['TEMP','SPD','SD','PCP06']]

In [304]:
df_avg=df.mean()

In [305]:
df_avg.columns=['temp_avg','spd_avg','snow_avg','rain_avg']

In [306]:
df_avg=np.round(df_avg,1)

In [307]:
#keep only months with at least 6 days to avoid anomalies (20%)
filt=df_avg.reset_index().groupby(['USAF','year','month'])[['day']].nunique()
filt2=filt[filt>5].dropna()

In [308]:
#keep only years with at least 3 months to avoid anomalies (20%)
filt3=filt2.reset_index().groupby(['USAF','year'])[['month']].nunique()
filt4=filt3[filt3>3].dropna()

In [309]:
df_avg2=df_avg.reset_index().set_index(['USAF','year'])

In [310]:
dfz={}

In [311]:
for i in filt4.index.unique(0):
    d=df_avg2.loc[i].loc[filt4.loc[i].index.unique()]
    dfz[i]=d
    d.to_csv('data/'+str(i)+'.csv')
    print(i)

119000
127560
127660
127720
127860
128050
128120
128150
128220
128250
128300
128305
128310
128360
128380
128390
128400
128430
128460
128470
128510
128550
128600
128603
128605
128660
128700
128820
128920
129100
129150
129200
129220
129250
129255
129300
129320
129350
129400
129410
129420
129500
129600
129700
129820
129920
150000
150001
150002
150010
150040
150070
150090
150100
150105
150140
150150
150200
150230
150235
150250
150320
150330
150400
150410
150420
150440
150470
150520
150550
150560
150630
150690
150730
150750
150800
150830
150850
150880
150890
150900
150940
150950
150990
151070
151080
151090
151110
151130
151170
151180
151190
151200
151205
151230
151240
151270
151320
151340
151360
151380
151400
151430
151450
151455
151480
151500
151540
151580
151590
151600
151620
151630
151650
151680
151700
151740
151790
151820
151840
151890
151940
151970
151990
152000
152005
152040
152060
152080
152090
152120
152150
152170
152190
152210
152300
152310
152350
152380
152410
152450
152470
152540

Load

In [312]:
import os
arr = os.listdir('data/')

In [313]:
for i in arr:
    if i not in ['all.csv']:
        dfz[i[:-4]]=pd.read_csv('data/'+i)
        print(i)

119000.csv
127560.csv
127660.csv
127720.csv
127860.csv
128050.csv
128120.csv
128150.csv
128220.csv
128250.csv
128300.csv
128305.csv
128310.csv
128360.csv
128380.csv
128390.csv
128400.csv
128430.csv
128460.csv
128470.csv
128510.csv
128550.csv
128600.csv
128603.csv
128605.csv
128660.csv
128700.csv
128820.csv
128920.csv
129100.csv
129150.csv
129200.csv
129220.csv
129250.csv
129255.csv
129300.csv
129320.csv
129350.csv
129400.csv
129410.csv
129420.csv
129500.csv
129600.csv
129700.csv
129820.csv
129920.csv
150000.csv
150001.csv
150002.csv
150010.csv
150040.csv
150070.csv
150090.csv
150100.csv
150105.csv
150140.csv
150150.csv
150200.csv
150230.csv
150235.csv
150250.csv
150320.csv
150330.csv
150400.csv
150410.csv
150420.csv
150440.csv
150470.csv
150520.csv
150550.csv
150560.csv
150630.csv
150690.csv
150730.csv
150750.csv
150800.csv
150830.csv
150850.csv
150880.csv
150890.csv
150900.csv
150940.csv
150950.csv
150990.csv
151070.csv
151080.csv
151090.csv
151110.csv
151130.csv
151170.csv
151180.csv

In [314]:
stationc=station.set_index(0)[2]

In [319]:
for z in dfz:
    dfz[z]=dfz[z].reset_index()

In [320]:
stationx={}
for z in dfz:
    stationx[stationc.loc[int(z)]]={'id':z,'yrs':len(dfz[z]['year'].unique())}

In [321]:
dq=pd.DataFrame(stationx).T.reset_index().set_index('yrs').sort_index(ascending=False)

In [322]:
dq.head(5)

,index,id
yrs,,
72,AUREL VLAICU,154200
68,GALATI,153100
67,CLUJ NAPOCA,151200
67,DROBETA TURNU SEVERIN,154100
66,CONSTANTA,154800


In [323]:
dds=[]
indicator='temp_avg'
for a in dq.head(500)['id'].values:
    dw=dfz[a]
    tmean=dw.groupby(['month','hour']).mean()[[indicator]]
    tmean.columns=['temp_mean']
    dw=dw.loc[1980:]
    dw=dw.loc[:2010]
    tmean80=dw.groupby(['month','hour']).mean()[[indicator]]
    tmean80.columns=['temp_mean80']
    dc=dfz[a].groupby(['year','month','hour']).mean()[[indicator]].join(tmean).join(tmean80)
    dc['temp_delta']=dc[indicator]-dc['temp_mean']
    dc['temp_delta80']=dc[indicator]-dc['temp_mean80']
    dd=dc.groupby(['year']).mean()
    dd=np.round(dd,1)
    dd['varos']=stationc[int(a)]
    dd['value']=1
    dds.append(dd)
ddb=pd.concat(dds)
ddb.to_csv('stripes/geo.csv')

In [324]:
dgeo=ddb.reset_index().set_index('varos').join(station.set_index(2))
dgeo[dgeo[3]=='HUNGARY'].to_csv('stripes/geotest.csv')

In [398]:
dds=[]
indicator='temp_avg'
# clean_months=[1,4,7,10]
clean_months=[1,2,3,4,5,6,7,8,9,10,11,12]
# clean_hours=[0,6,12,18]
clean_hours=[0,3,6,12,15,18,21]
# clean_hours=[6]
clean_slice=[(i,j) for i in clean_months for j in clean_hours]
for a in dq['id'].values:
    dw=dfz[a].groupby(['month','hour']).mean().loc[clean_slice]
    if dw[indicator].count()==len(clean_hours)*len(clean_months):
        
        dc=dfz[a].set_index(['month','hour']).loc[clean_slice].reset_index()
        dx=dc.groupby(['year']).nunique()
        full_data_years=dx[((dx['month']==len(clean_months))&(dx['hour']==len(clean_hours)))].index
        tmean=dw.groupby(['month','hour']).mean()[[indicator]]
        tmean.columns=['temp_mean']

        dw=dfz[a].set_index(['month','hour']).loc[clean_slice].reset_index().set_index(['year']).loc[full_data_years].reset_index()
        tmean_full_years=dw.groupby(['month','hour']).mean()[[indicator]]
        tmean_full_years.columns=['temp_mean_full_years']

        dc=dc.groupby(['year','month','hour']).mean()[[indicator]].loc[full_data_years].join(tmean).join(tmean_full_years)
        dc['temp_delta']=dc[indicator]-dc['temp_mean']     
        dc['temp_delta_full_years']=dc[indicator]-dc['temp_mean_full_years']     

        if ((2018 in dw['year'].unique()) and (2010 in dw['year'].unique())):
            tmean_last10=dw[dw['year']>2009].groupby(['month','hour']).mean()[[indicator]]
            tmean_last10.columns=['temp_mean_last10']
            dc=dc.join(tmean_last10)
            dc['temp_delta_last10']=dc[indicator]-dc['temp_mean_last10']
        
        dd=dc.groupby(['year']).mean()
        dd=np.round(dd,1)
        dd['varos']=stationc[int(a)]
        dd['value']=1
        dds.append(dd)
ddb=pd.concat(dds)
dgeo=ddb.reset_index().set_index('varos').join(station.set_index(2))
dgeo[dgeo[3]=='HUNGARY'].to_csv('stripes/hu.csv')
dgeo[dgeo[3]=='ROMANIA'].to_csv('stripes/ro.csv')
dgeo.to_csv('stripes/huro.csv')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  # Remove the CWD from sys.path while we load stuff.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:38: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [396]:
dds=[]
indicator='temp_avg'
clean_months=[1,5,9]
# clean_months=[1,2,3,4,5,6,7,8,9,10,11,12]
# clean_hours=[0,6,12,18]
# clean_hours=[0,3,6,12,15,18,21]
clean_hours=[6]
clean_slice=[(i,j) for i in clean_months for j in clean_hours]
for a in dq['id'].values:
    dw=dfz[a].groupby(['month','hour']).mean().loc[clean_slice]
    if dw[indicator].count()==len(clean_hours)*len(clean_months):
        
        dc=dfz[a].set_index(['month','hour']).loc[clean_slice].reset_index()
        dx=dc.groupby(['year']).nunique()
        full_data_years=dx[((dx['month']==len(clean_months))&(dx['hour']==len(clean_hours)))].index
        tmean=dw.groupby(['month','hour']).mean()[[indicator]]
        tmean.columns=['temp_mean']

        dw=dfz[a].set_index(['month','hour']).loc[clean_slice].reset_index().set_index(['year']).loc[full_data_years].reset_index()
        tmean_full_years=dw.groupby(['month','hour']).mean()[[indicator]]
        tmean_full_years.columns=['temp_mean_full_years']

        dc=dc.groupby(['year','month','hour']).mean()[[indicator]].loc[full_data_years].join(tmean).join(tmean_full_years)
        dc['temp_delta']=dc[indicator]-dc['temp_mean']     
        dc['temp_delta_full_years']=dc[indicator]-dc['temp_mean_full_years']     

        if ((2018 in dw['year'].unique()) and (2010 in dw['year'].unique())):
            tmean_last10=dw[dw['year']>2009].groupby(['month','hour']).mean()[[indicator]]
            tmean_last10.columns=['temp_mean_last10']
            dc=dc.join(tmean_last10)
            dc['temp_delta_last10']=dc[indicator]-dc['temp_mean_last10']
        
        dd=dc.groupby(['year']).mean()
        dd=np.round(dd,1)
        dd['varos']=stationc[int(a)]
        dd['value']=1
        dds.append(dd)
ddb=pd.concat(dds)
dgeo=ddb.reset_index().set_index('varos').join(station.set_index(2))
dgeo[dgeo[3]=='HUNGARY'].to_csv('stripes/hu_mini.csv')
dgeo[dgeo[3]=='ROMANIA'].to_csv('stripes/ro_mini.csv')
dgeo.to_csv('stripes/huro_mini.csv')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  # Remove the CWD from sys.path while we load stuff.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:38: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [397]:
station[station[2]=='DEBRECEN']

,0,1,2,3,4,5,6
28,128820,99999,DEBRECEN,HUNGARY,47.489,21.615,110.0


In [4]:
namer={"AGARD":"Agárd",
"AUREL VLAICU":"Bukarest - Aurel Vlaicu",
"BACAU":"Bákó",
"BAJA":"Baja",
"BEKESCSABA":"Békéscsaba",
"BOBOC AIR BASE":"Boboc légi bázis",
"BOTOSANI":"Botoșani",
"BUDAPEST/PESTSZENTLORINC":"Budapest - Pestszentlőrinc",
"BUZAU":"Buzău",
"CALARASI":"Călărasi",
"CAMPIA TURZII":"Aranyosgyéres",
"CARANSEBES":"Káránszebes",
"CATALOI":"Tulcea - Cataloi",
"CEAHLAU TOACA":"Csalhó",
"CLUJ NAPOCA":"Kolozsvár",
"CONSTANTA":"Konstanca",
"DEBRECEN":"Debrecen",
"DEVA":"Déva",
"DROBETA TURNU SEVERIN":"Szörényvár",
"EGER":"Eger",
"FERIHEGY":"Budapest - Ferihegy",
"GALATI":"Galac",
"GYOR":"Győr",
"HENRI COANDA":"Bukarest - Henri Coandă",
"IASI":"Jászvásár",
"JOSVAFO":"Jósvafő",
"KECSKEMET":"Kecskemét",
"KEKESTETO":"Kékestető",
"MIERCUREA CIUC":"Csíkszereda",
"MIHAIL KOGALNICEANU":"Konstanca - Mihail Kogălniceanu",
"MISKOLC":"Miskolc",
"MOSONMAGYAROVAR":"Mosonmagyaróvár",
"NAGYKANIZSA":"Nagykanizsa",
"NYIREGYHAZA":"Nyíregyháza",
"OCNA SUGATAG":"Aknasugatag",
"ORADEA":"Nagyvárad",
"PAKS":"Paks",
"PAPA":"Pápa",
"PECS SOUTH":"Pécs",
"POROSZLO":"Poroszló",
"RAMNICU VALCEA":"Râmnicu Vâlcea",
"ROSIORII DE VEDE":"Roșiorii De Vede",
"SARMELLEK":"Balaton - Sármellék",
"SATU MARE":"Szatmár",
"SIBIU":"Nagyszeben",
"SIOFOK":"Siófok",
"SOPRON":"Sopron",
"STEFAN CEL MARE":"Suceava - Ștefan Cel Mare",
"SULINA":"Sulina",
"SZECSENY":"Szécsény",
"SZEGED (AUT)":"Szeged",
"SZENTGOTTHARD/FARKASFA":"Szentgotthárd",
"SZOLNOK":"Szolnok",
"SZOMBATHELY ARPT / VAS":"Szombathely",
"TAT":"Tát",
"TATA":"Tata",
"TAUTII MAGHERAUS":"Nagybánya - Miszmogyorós",
"TRAIAN VUIA":"Temesvár - Traian Vuia",
"VARFU OMU":"Bucsecs - Omu csúcs",
"VESZPREM/SZENTKIRALYSZABADJA":"Veszprém",
"VIDRASAU":"Marosvásárhely - Vidrátszeg",
"ZAHONY":"Záhony",
"ZALAEGERSZEG/ANDRASHIDA":"Zalaegerszeg"}

In [5]:
import json

In [6]:
open('namer.json','w').write(json.dumps(namer))

2143